# Welcome to RunGPU Vision

This is a notebook to exclusively demonstrate the image model training capabilities that the Rungpu library offers. It's simple to use, has a plug-and-play interface. 

Image models can be used for several practices,but in this case we would be using Image models for training them to classify images 


In [46]:
from rungpu_vision import Model, Client, VisionDataset
from configparser import ConfigParser

### Enter your Rungpu Client_id and Client_Secret

In [47]:
client_id = "<Your Client ID Here>"
client_secret = "<Your Client Secret Here>"

In [48]:
config = ConfigParser()
config.read('system.ini')

client_id = config.get('rungpu_client','client_id')
client_secret = config.get('rungpu_client','client_secret')


## Create the Client Object for Authentication

In [49]:
client = Client(client_id, client_secret)

In [50]:
# Define a Dataset
dataset_config =  {"src_type": "kaggle", "src_id": "asdasdasasdas/garbage-classification"}
dataset = VisionDataset(client,dataset_config)



#### Create your Image dataset using the `create_dataset()` function

In [51]:
imgdat = dataset.create_dataset()
dataset_id = imgdat['dataset_id']
print(dataset_id)

imgdat_ed1d3557-68c9-4166-84ba-317172e69cc7


#### Image Dataset Status

Since these are image model, its a safe bet to keep checking the status of the image dataset ingestion into the RunGPU Platform since some image datasets might take time to download. 

Also, The `check_data` gives you a `dir_map` feature, which let's you narrow out which specific directory has the right image-label mappings you would want to use for your classification model, and you can use that specific path in the `dataset_path` in the model configuration.

In [60]:
dataset.check_data(dataset_id)

{'Dataset ID': 'imgdat_ed1d3557-68c9-4166-84ba-317172e69cc7',
 'dataset_status': {'command': 'Vision',
  'status': 'COMPLETED',
  'dataset_id': 'imgdat_ed1d3557-68c9-4166-84ba-317172e69cc7',
  'dir_map': {'Files': {'garbage_classification': 5,
    'Garbage_classification': 5},
   'garbage_classification': {'Garbage_classification': 0},
   'garbage_classification/Garbage_classification': {'glass': 0,
    'paper': 0,
    'trash': 0,
    'metal': 0,
    'cardboard': 0,
    'plastic': 0},
   'garbage_classification/Garbage_classification/glass': {},
   'garbage_classification/Garbage_classification/paper': {},
   'garbage_classification/Garbage_classification/trash': {},
   'garbage_classification/Garbage_classification/metal': {},
   'garbage_classification/Garbage_classification/cardboard': {},
   'garbage_classification/Garbage_classification/plastic': {},
   'Garbage_classification': {'Garbage_classification': 0},
   'Garbage_classification/Garbage_classification': {'glass': 0,
    'pa

### Define the model training configuration

In [61]:
# Define a Config

config = {
    "model": "<name of the pretrained model you want to use>",
    # "dataset": "<path-for-the-dataset>"
    "n_classes": "<Number of classes",
    "optimizer" : "<optimizer-for-training>",
    "learning_rate" : "<learning rate used during training>",
    "momentum": "<momentum used during training>",
    "metrics" : "<metrics for training>",
    "epochs": "<number of epochs you want to train for>",
    "batch_size": "<int batch size for training>",
    "layers": [
    # Stack your layers up in this list in the following format
        #(layer_name: <name_of_the_layer, layer_type: <CONV2D | RELU | FC>, dim:(20,20,..)),
        #(layer_name: <name_of_the_layer, layer_type: <CONV2D | RELU | FC>, dim:(20,20,..))
    ]
        

}

In [62]:
config = {
        
        "model": "resnet50",
        "optimizer": "SGD",
        "n_classes": 6,
        "learning_rate": 0.001,
        "labels":  {"cardboard": 0, "glass": 1,"metal": 2, "paper": 3,"plastic": 4, "trash": 5},
        "dataset_id": dataset_id,
        "dataset_path": "garbage_classification/Garbage_classification",
        "momentum": 0.9,
        "metrics": ['accuracy'],
        "loss": "sparse_categorical_entropy",
        "epochs": 10, 
        "layers": [('lin1',"FC",(2048,1080)), ('sig1',"SIG"),('lin2',"FC",(1080,6)),('relu1',"RELU")], 
        "batch_size": 16
}

# Create the Model

In [63]:
model = Model(client=client,config=config)


## Run the Model 

In [64]:
model = model.train_model()

In [65]:
model

{'command': 'Vision',
 'status': 'IN QUEUE',
 'phase': 'TRAINING',
 'train_id': 'resnet50-80ea3b74-cce4-4350-bc85-6193d921b29d',
 'run_history': 'null',
 'dataset_id': 'imgdat_ed1d3557-68c9-4166-84ba-317172e69cc7',
 'run_start': None,
 'run_end': None,
 'error': 'Job Still in Queue',
 'config': {'client_id': 'n6p7iWSrknJqkLIwX0PGi',
  'client_secret': '3b32447eab2155d1905c44dcf0506e5ceddf3827b',
  'model': 'resnet50',
  'dataset_id': 'imgdat_ed1d3557-68c9-4166-84ba-317172e69cc7',
  'dataset_path': 'garbage_classification/Garbage_classification',
  'n_classes': 6,
  'optimizer': 'SGD',
  'epochs': 10,
  'batch_size': 16,
  'learning_rate': 0.001,
  'momentum': 0.9,
  'loss': 'sparse_categorical_entropy',
  'metrics': ['accuracy'],
  'layers': [['lin1', 'FC', [2048, 1080]],
   ['sig1', 'SIG'],
   ['lin2', 'FC', [1080, 6]],
   ['relu1', 'RELU']],
  'labels': {'cardboard': 0,
   'glass': 1,
   'metal': 2,
   'paper': 3,
   'plastic': 4,
   'trash': 5}}}

In [66]:
# Get your model id

model_id = model['train_id']
model_id

'resnet50-80ea3b74-cce4-4350-bc85-6193d921b29d'

## Let's check the status of your running Job

In [67]:
from rungpu_vision import VisionStatus

status = VisionStatus(client, model_id)


##### Call the `get_status()` method on the status object to get the current training status of the job

In [75]:
status.get_status()


{'Train_Id': 'resnet50-80ea3b74-cce4-4350-bc85-6193d921b29d',
 'time_elapsed': '2.85116975',
 'train_status': {'command': 'Vision',
  'status': 'COMPLETED',
  'phase': 'MODEL_TRAINED',
  'model_id': 'resnet50-80ea3b74-cce4-4350-bc85-6193d921b29d',
  'dataset_created': 'NO',
  'run_history': 'null',
  'dataset_id': 'imgdat_ed1d3557-68c9-4166-84ba-317172e69cc7',
  'run_start': '2024-08-22 15:38:04.124711',
  'run_end': '2024-08-22 15:40:55.194896',
  'error': 'No Errors',
  'process_id': '2653276',
  'config': {'client_id': 'n6p7iWSrknJqkLIwX0PGi',
   'client_secret': '3b32447eab2155d1905c44dcf0506e5ceddf3827b',
   'model': 'resnet50',
   'dataset_id': 'imgdat_ed1d3557-68c9-4166-84ba-317172e69cc7',
   'dataset_path': 'garbage_classification/Garbage_classification',
   'n_classes': 6,
   'optimizer': 'SGD',
   'epochs': 10,
   'batch_size': 16,
   'learning_rate': 0.001,
   'momentum': 0.9,
   'loss': 'sparse_categorical_entropy',
   'metrics': ['accuracy'],
   'layers': [['lin1', 'FC', [